# Neural networks

In this exercise, we will write a neural network with one hidden layer for a multi-class classification problem. We are only concerned about the functionality, not the performance. 

Please check the plot below for a sketch of the NN structure and the naming conventions
![](NN_structure1.png)

In [25]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from IPython.display import IFrame

In [26]:
def sigmoid(z):
    return 1/(1.0+np.exp(-z))

Implement the softmax function which is used for multi-class classification problems. The function converts the outputs of the last layer of a neural network to probabilities of belonging to one of the classes in the classification. It takes a `nd x nc` matrix as input where `nd` is the number of data points and `nc` the number of classes.

In [27]:
def softmax(x):
    # YOUR CODE HERE
    answer = []
    for X in x:
        answerTmp = []
        total = sum(np.exp(X))
        for z in X:
            answerTmp.append(np.exp(z) / total)
        answer.append(answerTmp)
    return np.array(answer)

In [28]:
assert np.isclose(softmax(np.array([[400,300,100]])), np.array([[1.00000000e+000, 3.72007598e-044, 5.14820022e-131]])).all()

In [29]:
ztest = np.array([
    [10,-10,-10,-10,-10],
    [ 0.47227959,  0.08658337,  0.08738361,  0.14692136, -0.26323141]  
])

assert softmax(ztest).shape == ztest.shape
assert np.isclose( softmax(ztest),  np.array([[9.99999992e-01, 2.06115361e-09, 2.06115361e-09, 2.06115361e-09,
        2.06115361e-09],
       [2.80738986e-01, 1.90896070e-01, 1.91048893e-01, 2.02768946e-01,
        1.34547105e-01]])).all()

This is the inputs we will use to test our implementation. 

In [30]:
ni = 4
nh = 5
no = 7
nd = 10
np.random.seed(1212)
v = np.random.random(size=(nh,ni+1))
iin = np.random.random(size=(nd,ni))
w = np.random.random(size=(no,nh+1))
xin = np.random.random(size=(nd,nh))

ys = [ [1],[0],[2],[2],[3],[4],[1],[6],[5],[4]]
enc = OneHotEncoder(sparse=False, categories='auto')
yij = enc.fit_transform(ys)



Define the function `forwardPass` that calculates the output of a network layer with parameters `w` and inputs `inp`, a `nd x nf` matrix of `nd` data samples with `nf` input features. Use the `sigmoid` function (defined above) as your activation function. 

In [31]:
def forwardPass(inp,w):
    inp = np.insert(inp, 0, values=np.ones(np.shape(inp)[0]), axis=1)
    return sigmoid(inp.dot(w.T))

In [32]:
assert forwardPass(iin,v).shape == (nd, nh)
assert np.isclose(forwardPass(iin,v),
        np.array([[0.68837646, 0.7968613 , 0.79800127, 0.65126304, 0.7655118 ],
       [0.81273724, 0.89213343, 0.86406507, 0.81389362, 0.86413682],
       [0.82533989, 0.90025807, 0.84266121, 0.81861927, 0.85204489],
       [0.82041648, 0.86399255, 0.86640616, 0.77089753, 0.85294415],
       [0.82014636, 0.86158128, 0.80726324, 0.7968344 , 0.81375178],
       [0.8229169 , 0.89412018, 0.86592284, 0.80538115, 0.86451929],
       [0.85160111, 0.87919916, 0.8167712 , 0.82548824, 0.83057137],
       [0.74336211, 0.86585661, 0.84263023, 0.73507403, 0.82932382],
       [0.8342882 , 0.91019903, 0.91391614, 0.80546577, 0.90408941],
       [0.71588926, 0.87475359, 0.92144777, 0.68106143, 0.89086182]])).all()

Define the function `forwardPassSoftmax` that calculates the output of a network layer with parameters `w` and inputs `inp`, a `nd x nf` matrix of `nd` data samples with `nf` input features. Use the `softmax` function (defined above) as your activation function. 

In [33]:
def forwardPassSoftmax(inp,w):
    inp = np.insert(inp, 0, values=np.ones(np.shape(inp)[0]), axis=1)
    return softmax(inp.dot(w.T))

In [34]:
hout = forwardPass(iin,v)
assert forwardPassSoftmax(hout,w).shape == (nd, no)
assert np.isclose(forwardPassSoftmax(hout,w),np.array([[0.07130809, 0.12409133, 0.11275842, 0.15968169, 0.16715723,
        0.26521125, 0.099792  ],
       [0.06725052, 0.12112668, 0.11300347, 0.15521793, 0.1656938 ,
        0.28376125, 0.09394635],
       [0.06783011, 0.12054124, 0.11408981, 0.15484997, 0.16421653,
        0.28325473, 0.0952176 ],
       [0.06792065, 0.12228852, 0.11538123, 0.15665073, 0.1683589 ,
        0.27616102, 0.09323895],
       [0.06888607, 0.12115008, 0.11676099, 0.15501673, 0.16230241,
        0.2786129 , 0.09727082],
       [0.06747572, 0.12117407, 0.11375229, 0.15555633, 0.16662511,
        0.28177475, 0.09364173],
       [0.06825527, 0.12057384, 0.1174965 , 0.15421526, 0.16208508,
        0.28121404, 0.09616001],
       [0.06903226, 0.12228289, 0.11112691, 0.15762792, 0.16737128,
        0.27613176, 0.09642699],
       [0.06636304, 0.1215225 , 0.11244603, 0.15617258, 0.1706804 ,
        0.28243318, 0.09038227],
       [0.06757224, 0.12412115, 0.10706434, 0.16057489, 0.17650805,
        0.27296254, 0.09119679]])).all()

We will check that our implementation works by comparing with `sklearn`'s implementation. We are not interested in the actual fit, but we need to call the fit function so that we have a working model.

In [35]:
from sklearn.neural_network import MLPClassifier
classifier = MLPClassifier(
    random_state=1,
    hidden_layer_sizes=(nh,),
    max_iter=10,
    activation='logistic',
)
classifier.fit(iin,yij)

/opt/conda/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPClassifier(activation='logistic', alpha=0.0001, batch_size='auto',
              beta_1=0.9, beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(5,), learning_rate='constant',
              learning_rate_init=0.001, max_iter=10, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=1, shuffle=True, solver='adam', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

In [36]:
w1,w2 = classifier.coefs_
w1, w2 = np.array(w1), np.array(w2)
i1,i2 = classifier.intercepts_

# combine the coefs and intercept values to weight vectors
w1ni, w1no = w1.shape
w1all = np.empty((w1no, w1ni+1))
w1all[:,0] = i1
w1all[:,1:] = w1.T
w2ni, w2no = w2.shape
w2all = np.empty((w2no, w2ni+1))
w2all[:,0] = i2
w2all[:,1:] = w2.T

mySolution = forwardPassSoftmax(forwardPass(iin,w1all),w2all)
classifier.out_activation_ = 'softmax'
skSolution = classifier.predict_proba(iin)

assert np.isclose(skSolution, mySolution).all()

Define the following loss functions: 
- `Jyhat` is the loss as a function of the outputs of the output layer (the one with the softmax step.) 
- `Jx` is the loss as a function of the outputs of the hidden layer. 
- `J` is the loss as a function of the network input `I`, the parameters of the input-hidden connections `v`, the parameters of the hidden-output connections `w` and the one-hot encoded targets `yij`. 

In [37]:
# yij: true one-hot encoded classification of the input sample
# yhat: outputs of the NN's output layer, including softmax
def Jyhat(yhat,yij):
    answer = 0
    for i in range(len(yhat)):
        for j in range(len(yhat[0])):
            answer += yij[i][j] * np.log(yhat[i][j])
    return -answer

In [38]:
yhat = forwardPassSoftmax(forwardPass(iin,v),w)
Jyhat(yhat,yij)
assert np.isclose(Jyhat(yhat,yij) , 20.225746159683087)

In [39]:
# yij: true one-hot encoded classification of the input sample
# X: inputs for the output layer (= outputs of the hidden layer)
# w: weights for the conntection hidden-output
def Jx(X,w,yij):
    yhat = forwardPassSoftmax(X, w)
    return Jyhat(yhat, yij)
    # YOUR CODE HERE

In [40]:
x = forwardPass(iin,v)
Jx(x,w2all,yij)
assert np.isclose(Jx(x,w,yij) , 20.225746159683087)

In [41]:
# yij: true one-hot encoded classification of the input sample
# w: weights for the current layer
# I: network input 
# v: weight of the connection input-hidden
def J(I,v,w,yij):
    yhat = forwardPassSoftmax(forwardPass(iin,v),w)
    return Jyhat(yhat, yij)
    # YOUR CODE HERE

In [42]:
J(iin,w1all,w2all,yij)
assert np.isclose(J(iin,v,w,yij) , 20.225746159683087)

## Derivatives of the loss function

We will now define different derivatives of the loss function. They will be useful for implementing gradient descent algorithm which updates the weights based on the derivatives of the loss function with respect to the weights. 

We will check our result numerically.

Define the derivative of the softmax function with respect to its arguments. If should return a `n x n` matrix, where `n` is the length ot the vector of arguments `z` 

In [43]:
def ds_dz(z):
    answer = []
    now = softmax([z])
    for i in range(len(z)):
        temp = []
        for j in range(len(z)):
            if i == j:
                temp.append(now[0][i] * (1 - now[0][j]))
            else:
                temp.append(now[0][i] * (-1 * now[0][j]))
        answer.append(temp)
    return answer

This checks the implementation of the derivative.

In [44]:
eps = 1e-8
z = np.array([[0.1,0.2,0.3]])
analytical = ds_dz(z[0])

z2 = np.array([[0.1+eps,0.2,0.3]])
numerical = (softmax(z2)-softmax(z))/eps

assert np.isclose(numerical[0],analytical[0]).all() 

z2 = np.array([[0.1,0.2+eps,0.3]])
numerical = (softmax(z2)-softmax(z))/eps

assert np.isclose(numerical[0],analytical[1]).all() 

z2 = np.array([[0.1,0.2,0.3+eps]])
numerical = (softmax(z2)-softmax(z))/eps

assert np.isclose(numerical[0],analytical[2]).all() 

Define the function `dJ_dy` which is the derivative of the loss function with respect to the output of the softmax function. It should be a `nd x no` matrix (`no` output values for each of the `nd` events in our data sample). 

In [45]:
def dJ_dyhat(yhat, yij):
    answer = []
    for i in range(len(yhat)):
        temp = []
        for j in range(len(yhat[0])):
            temp.append(-1 * yij[i][j] / yhat[i][j])
        answer.append(temp)
    return np.array(answer)
    # YOUR CODE HERE

In [46]:
yhat = forwardPassSoftmax(forwardPass(iin,v),w)
assert dJ_dyhat(yhat,yij).shape == (nd, no)

analytical = np.sum(dJ_dyhat(yhat, yij),axis=0)

eps = 1e-6
for i in range(no):
    y2 = np.array(yhat)
    y2[:,i] += eps
    num = (Jyhat(y2,yij)-Jyhat(yhat,yij))/eps
    assert np.isclose(num,analytical[i])

Define the function `dJ_dwij` which is the derivative of the loss function with respect to the weights between the hidden layer and the output layer. It should be a `nh+1 x no` matrix.

In [51]:
def dJ_dwij(xin,w,yij):    

    nd, nh = xin.shape
    _, no = yij.shape
    yhat = forwardPassSoftmax(xin,w)
    
    Xaug = np.empty( (nd, nh+1)) # nd x (nf+1)
    Xaug[:,1:] = xin
    Xaug[:,0] = np.ones(nd)
    z = np.dot(Xaug,w.T)
    s = softmax(z)
    res = np.zeros( (nh+1, no))
    
    for k in range(nh + 1):
        for m in range(no):
            answerTemp = 0
            for l in range(no):
                for i in range(nd):
                    if l == m:
                        answerTemp += (-1 * yij[i][l] / yhat[i][l]) * s[i][l] * (1 - s[i][m]) * Xaug[i][k]
                    else:
                        answerTemp += (-1 * yij[i][l] / yhat[i][l]) * s[i][l] * (-1 * s[i][m]) * Xaug[i][k]
            res[k,m] = answerTemp
    return res


In [52]:
analytic = dJ_dwij(xin,w,yij)

assert analytic.shape == (nh+1, no)

for i in range(nh+1):
    for j in range(no):
        w2 = np.array(w)
        w2[j,i] += eps 
        num = (Jx(xin,w2,yij)-Jx(xin,w,yij))/eps
        assert np.isclose(analytic[i,j],num, rtol=1e-4)

Define the function `dJ_dx` that is the derivative of the loss function with respect to the activation values of the hidden layer. It should be a `nd x nh` matrix. 

In [53]:
def dJ_dx(xin,w,yij):
    nd, nh = xin.shape
    Xaug = np.empty( (nd, nh+1))
    Xaug[:,1:] = xin
    Xaug[:,0] = np.ones(nd)
    z = np.dot(Xaug,w.T)
    _, nc = yij.shape
    yhat = forwardPassSoftmax(xin,w)

    dj_dx = np.zeros((nd,nh))
    
    # YOUR CODE HERE
    for i in range(nd):
        for j in range(nh):
            sum = 0
            for x in range(len(w)):
                for y in range(len(w[0])):
                    
                    
        dj_dx = 
    return dj_dx


In [55]:
analytical = dJ_dx(xin, w, yij)
assert analytical.shape == (nd, nh)
eps = 1e-7
for i in range(ni):
    for d in range(nd):
        xin2 = np.array(xin)
        xin2[d,i] += eps 
        num = (Jx(xin2,w,yij)-Jx(xin,w,yij))/eps
        print(num)

-0.01105441072013491
-0.14158246131046326
-0.6121638307376998
-0.582240176072446
0.0013205436744101462
-0.12660425596777714
0.06885166214942728
0.27235124377966713
0.26624860538504436
-0.15096642158596296
0.2829203893384147
-0.28666384821463
0.3029994743997122
0.2950542921098531
-0.035356961802790465
-0.101378745398506
0.24324936731545677
-0.23446816044270236
-0.021765735880308057
-0.07728317541477736
0.020403270184488065
-0.10991573873297966
0.013407692733835574
0.014791439184591582
-0.053006097289198806
-0.4201946879334173
0.025477859821876336
0.4000284903327156
0.10906887837336399
-0.45246757451877784
0.1097927082582828
0.322221147541768
0.03579796015173997
0.02830056189395691
0.26039074896289094
0.3933003611678032
0.07248502242873656
-0.1495627088843321
-0.465654963477391
0.40749242202764435


In [54]:
analytical = dJ_dx(xin, w, yij)
assert analytical.shape == (nd, nh)
eps = 1e-7
for i in range(ni):
    for d in range(nd):
        xin2 = np.array(xin)
        xin2[d,i] += eps 
        num = (Jx(xin2,w,yij)-Jx(xin,w,yij))/eps
        assert (np.isclose(analytical[d,i], num, rtol=1e-4))

AssertionError: 

Define the function `dJ_dvij` which is the derivative of the loss function with respect to the weights between the hidden layer and the output layer. It should be a `ni+1 x nh` matrix.

In [ ]:
def dJ_dvij(iin,v,w,yij):    
    nh, _ = v.shape
    xin = forwardPass(iin,v)
    nd, ni = iin.shape

    Xaug = np.empty( (nd, ni+1)) # nd x (nf+1)
    Xaug[:,1:] = iin
    Xaug[:,0] = np.ones(nd)
    z = np.dot(Xaug,v.T)
    y = sigmoid(z)

    res = np.zeros((ni+1,nh))

    # YOUR CODE HERE
    return res


In [ ]:
analytic = dJ_dvij(iin,v,w,yij)
assert analytic.shape == (ni+1, nh)

eps = 1e-7
for i in range(ni+1):
    for j in range(nh):
        v2 = np.array(v)
        v2[j,i] += eps 
        ndiff = (J(iin,v2,w,yij)-J(iin,v,w,yij))/eps
        assert np.isclose(analytic[i,j], ndiff,rtol=1e-4)

# Application

Here is another set of Iris data. We will use our new implementation of a neural network to fit classifier to this data.

In [ ]:
from sklearn import datasets
from sklearn.preprocessing import StandardScaler


data_dic = datasets.load_iris()
features = data_dic['data']
targets = data_dic['target']

c1 = features[targets==0]
c2 = features[targets==1]
c3 = features[targets==2]

ind1, ind2 = 0,2
plt.scatter(c1[:,ind1],c1[:,ind2], color='red', marker='s', alpha=0.5, label="Setosa")
plt.scatter(c2[:,ind1],c2[:,ind2], color='blue', marker='x', alpha=0.5, label="Versicolour")
plt.scatter(c3[:,ind1],c3[:,ind2], color='green', marker='o', alpha=0.5, label="Versicolour")
plt.legend()
plt.xlabel("sepal length [cm]")
plt.ylabel("sepal width [cm]");
plt.savefig('iris_3class.png')
def subSample(nData):
    X = np.empty((3*nData,2))
    X[:nData] = c1[:nData,(ind1, ind2)]
    X[nData:2*nData] = c2[:nData,(ind1, ind2)]
    X[2*nData:] = c3[:nData,(ind1, ind2)]
    Y = np.empty(3*nData)
    Y[:nData] = np.zeros(nData)
    Y[nData:2*nData] = np.ones(nData)
    Y[2*nData:] = 2*np.ones(nData)
    return X,Y

X, Y = subSample(50)
scaler = StandardScaler()
X = scaler.fit_transform(X)

We will use a neural network with 1 hidden layer with 3 units to classify the three classes. 

In [ ]:
ni = 2
nh = 3
no = 3
nd = len(X)

np.random.seed(1211)
v = np.random.random(size=(nh,ni+1))
w = np.random.random(size=(no,nh+1))

enc = OneHotEncoder(sparse=False, categories='auto')
yij=enc.fit_transform(Y[:,np.newaxis])


Implement gradient descent to train this network, i.e. update the weights according to the derivative of the loss function. Use 10000 steps with learning rate 0.05. 

In [ ]:
eta = 0.05

for i in range(100):
    # YOUR CODE HERE

This code will display the result of the training

In [ ]:
def predict(X):
    xs = forwardPass(X,v)
    ys = forwardPass(xs,w)
    probs = softmax(ys)
    return np.argmax(probs,axis=1)

from matplotlib.colors import ListedColormap

def regions3(X, y,colors,resolution=0.02, savename=None):
    # setup marker generator and color map
    markers = ('s', 'x', 'o', '^', 'v')
    cmap = ListedColormap(colors[:len(np.unique(y))])
    # plot the decision surface
    x1_min, x1_max = X[:, 0].min() - 0.1, X[:, 0].max() + 0.1
    x2_min, x2_max = X[:, 1].min() - 0.1, X[:, 1].max() + 0.1
    xx1, xx2 = np.meshgrid(np.arange(x1_min, x1_max, resolution),
                            np.arange(x2_min, x2_max, resolution))
    Z = predict(np.array([xx1.ravel(), xx2.ravel()]).T)
    Z = Z.reshape(xx1.shape)
    plt.contourf(xx1, xx2, Z, alpha=0.1, cmap=cmap)
    plt.xlim(xx1.min(), xx1.max())
    plt.ylim(xx2.min(), xx2.max())
    
    c2 = X[y==2]
    c1 = X[y==1]
    c0 = X[y==0]
    
    xb,yb=c0[:,0],c0[:,1]
    plt.scatter(xb,yb,color=colors[0],alpha=0.4)
    xb,yb=c1[:,0],c1[:,1]
    plt.scatter(xb,yb,color=colors[1],alpha=0.4)
    xb,yb=c2[:,0],c2[:,1]
    plt.scatter(xb,yb,color=colors[2],alpha=0.4)
    if savename:
        plt.savefig(savename)
        
regions3(X, Y,['red','blue','green'],resolution=0.01)